<a href="https://colab.research.google.com/github/Peacewh/Peacewh/blob/main/notebooks/CSC309_Week05_KRR_Logic_Student_Centred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CSC309 – Artificial Intelligence  
**Week 5 Lab:** Knowledge Representation & Reasoning — Logic & Inference

**Instructor:** Dr Sakinat Folorunso

**Title:** Associate Professor of AI Systems and FAIR Data **Department:** Computer Sciences, Olabisi Onabanjo University, Ago-Iwoye, Ogun State, Nigeria

**Course Code:** CSC 309

**Mode:** Student‑centred, hands‑on in Google Colab

> Every code cell is commented line‑by‑line so you can follow the logic precisely.

## How to use this notebook
1. Start with the **Group Log** and **Do Now**.  
2. Run the **Setup** cell once.  
3. Work through **Tasks**. Edit only cells marked **`# TODO(Student)`**.  
4. Use **Quick Checks** to test your understanding.  
5. Finish with the **Reflection**. If you finish early, try the **Extensions**.

In [3]:
#@title 🧑🏽‍🤝‍🧑🏾 Group Log (fill before you start)
# The '#@param' annotations create form fields in Colab for easy input.

group_members = "Type names here"  #@param {type:"string"}  # Names of teammates
roles_notes = "Driver/Navigator, decisions, questions"  #@param {type:"string"}  # Short working notes

print("👥 Group:", group_members)        # Echo the group list for confirmation
print("📝 Notes:", roles_notes)          # Echo the notes so they're preserved in output

👥 Group: Type names here
📝 Notes: Driver/Navigator, decisions, questions


### Learning Objectives
- Encode facts/rules in a small **logic knowledge base**.  
- Query using unification; practice NL→logic, CNF/resolution (by hand).

In [4]:
#@title 🔧 Setup
# We try to use 'kanren' for a taste of relational programming.
# If unavailable, we fall back to a tiny custom rule demo.

import sys, subprocess                                     # Tools for optional installs
def pip_install(pkgs):
    for p in pkgs:
        try: __import__(p.split("==")[0])                  # Try importing
        except Exception:
            subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", p])  # Install if missing

try:
    pip_install(["kanren", "unification", "toolz"])        # Attempt to install kanren ecosystem
    from kanren import Relation, facts, run, var           # Import relational primitives
    using_kanren = True                                     # Flag to indicate the backend in use
except Exception as e:
    print("Using simple fallback (kanren not available):", e) # Inform the student
    using_kanren = False                                    # Switch to fallback
print("✅ Setup complete for Week 5.")

✅ Setup complete for Week 5.


In [5]:
#@title 📐 Starter KB (line‑by‑line comments)

if 'using_kanren' in globals() and using_kanren:    # If kanren is available
    parent = Relation()                              # Declare a binary relation 'parent(x,y)'
    facts(parent, ("ada","sam"), ("sam","tayo"), ("sam","ife"), ("ife","bola"))  # Add some facts
    x = var()                                        # Logic variable 'x'
    y = var()                                        # Logic variable 'y'
    # Query: who are Ada's grandchildren?
    result = run(0, x, parent("ada", x), parent(x, y))   # Find x such that parent(ada,x) and parent(x,y)
    print("Grandchildren of Ada (via kanren):", set(result))  # Show solutions
else:
    # Fallback: a tiny Python-based relation
    PARENT = {("ada","sam"), ("sam","tayo"), ("sam","ife"), ("ife","bola")}  # Parent relation as a set of tuples
    def grandchildren(p):                 # Compute grandchildren of 'p' by relational join
        return {c2 for (p1,c1) in PARENT for (p2,c2) in PARENT if p1==p and p2==c1}
    print("Grandchildren of Ada (fallback):", grandchildren("ada"))  # Show result

Grandchildren of Ada (via kanren): {'sam'}


In [ ]:
# TODO(Student): Encode 'All humans are mortal; Socrates is human; therefore Socrates is mortal.'
# Option A (kanren): Create relations HUMAN(x) and MORTAL(x); add facts and rules; run a query for Socrates.
# Option B (fallback): Represent as Python sets/dicts and derive 'mortal(socrates)' procedurally.

Option A — kanren logic-programming version

In [12]:
if 'using_kanren' in globals() and using_kanren:
    from kanren import Relation, facts, var, run

    # Declare relations
    HUMAN = Relation()
    # MORTAL = Relation() # Not strictly needed for this query as rule is embedded

    # Facts
    facts(HUMAN, ("socrates",))

    # Rule: All humans are mortal → mortal(x) :- human(x)
    # In kanren, rules are expressed through goals inside the query,
    # so we don’t "store" the rule; we express it while querying.

    x = var()

    # Query for: mortal(socrates)?
    # To find if Socrates is mortal, we check if Socrates is human.
    # If Socrates is human, then by the rule, Socrates is mortal.
    # So, we query if Socrates is a human.
    result = run(1, x, HUMAN(x))

    # Check if 'socrates' is among the humans found by the query
    print("Is Socrates mortal (via kanren)?", ("socrates" in result))

else:
    # Represent humans and mortals as sets
    HUMAN = {"socrates"}
    MORTAL = set()

    # Rule: All humans are mortal
    for h in HUMAN:
        MORTAL.add(h)

    print("Is Socrates mortal (fallback)?", "socrates" in MORTAL)



Is Socrates mortal (via kanren)? True


### Quick Check
Translate **two** everyday constraints into predicate logic, then back into English.  
E.g., “All 300‑level courses have a CA in Week 7.”

✅ Constraint 1
English:

“All hostel residents must pay their accommodation fee.”

Predicate Logic:

Let

𝑅
𝑒
𝑠
𝑖
𝑑
𝑒
𝑛
𝑡
(
𝑥
)
Resident(x) = “x lives in the hostel”

𝑃
𝑎
𝑖
𝑑
(
𝑥
)
Paid(x) = “x has paid the accommodation fee”

∀
𝑥

(
𝑅
𝑒
𝑠
𝑖
𝑑
𝑒
𝑛
𝑡
(
𝑥
)
→
𝑃
𝑎
𝑖
𝑑
(
𝑥
)
)
∀x(Resident(x)→Paid(x))

In [13]:
from kanren import Relation, facts, var, run

# Predicates
RESIDENT = Relation()
PAID = Relation()

# Facts
facts(RESIDENT, ("peace",), ("tola",), ("sam",))
facts(PAID, ("peace",), ("sam",))
# Tola has not paid (to test rule)

x = var()

# Check compliance: For each resident x, does PAID(x) hold?
residents = run(0, x, RESIDENT(x))

print("Accommodation Fee Compliance:")
for r in residents:
    has_paid = bool(run(1, x, PAID(r)))
    print(f"{r}: paid? {has_paid}")


Accommodation Fee Compliance:
peace: paid? True
sam: paid? True
tola: paid? False


✅ Constraint 2
English:

“Every class scheduled for Monday must have an attendance record.”

Predicate Logic:

Let

𝑀
𝑜
𝑛
𝑑
𝑎
𝑦
(
𝑐
)
Monday(c) = “class c is on Monday”

𝐴
𝑡
𝑡
𝑒
𝑛
𝑑
𝑎
𝑛
𝑐
𝑒
(
𝑐
)
Attendance(c) = “attendance record exists for c”

∀
𝑐

(
𝑀
𝑜
𝑛
𝑑
𝑎
𝑦
(
𝑐
)
→
𝐴
𝑡
𝑡
𝑒
𝑛
𝑑
𝑎
𝑛
𝑐
𝑒
(
𝑐
)
)
∀c(Monday(c)→Attendance(c))

In [14]:
from kanren import Relation, facts, var, run

# Predicates
MONDAY = Relation()
ATTENDANCE = Relation()

# Facts
facts(MONDAY, ("math101",), ("cs201",))
facts(ATTENDANCE, ("math101",))
# cs201 has no attendance yet

c = var()

monday_classes = run(0, c, MONDAY(c))

print("\nAttendance Compliance for Monday Classes:")
for cls in monday_classes:
    has_attendance = bool(run(1, c, ATTENDANCE(cls)))
    print(f"{cls}: attendance recorded? {has_attendance}")



Attendance Compliance for Monday Classes:
math101: attendance recorded? True
cs201: attendance recorded? False
